In [2]:
from datetime import datetime, date
from functools import reduce

import pandas as pd
import numpy as np

import math
import os


# Goal of this project is to prepare the assessment of the following question stated below.

## Within a district, is there a correlation between the time taken to complete a service request and the total crime in that area? If so, is it possible to predict “crime” based on the district’s service requests? 

# Begin processing Income Dataframe

In [11]:
income_df = pd.read_csv("../datasets/Vital_Signs_16_Census_Demographics.csv")

In [12]:
income_neighborhood = []
income_total_pop = []
income_medium_hh = []

for index, row in income_df.iterrows():
    if "/" in row["CSA2010"]:        
        temp_districts = row["CSA2010"].split("/")
        
        for dist in temp_districts:
            income_neighborhood.append(dist.lower())
            income_total_pop.append(row["tpop10"])
            income_medium_hh.append(row["mhhi16"])
    else:
        income_neighborhood.append(row["CSA2010"].lower())
        income_total_pop.append(row["tpop10"])
        income_medium_hh.append(row["mhhi16"])
    

processed_income = {
    'Neighborhood': income_neighborhood,
    'Total Population': income_total_pop,
    'Median Household Income': income_medium_hh
}

processed_income_df = pd.DataFrame(data=processed_income)

In [13]:
processed_income_df.head(10)

,Median Household Income,Neighborhood,Total Population
0,37302.17105,allendale,16217
1,37302.17105,irvington,16217
2,37302.17105,s. hilton,16217
3,53565.07970,beechfield,12264
4,53565.07970,ten hills,12264
5,53565.07970,west hills,12264
6,40482.35965,belair-edison,17416
7,38603.93023,brooklyn,14243
8,38603.93023,curtis bay,14243
9,38603.93023,hawkins point,14243


In [14]:
unique_income_neighborhoods = np.unique(processed_income_df["Neighborhood"])

# Begin processing Crime Dataframe

In [15]:
crime_df = pd.read_csv("../datasets/BPD_Part_1_Victim_Based_Crime_Data.csv")

In [16]:
start = datetime(2016, 1, 1, 0, 0, 0)
end = datetime(2016, 12, 31, 23, 59, 59)

print(start)
print(end)

2016-01-01 00:00:00
2016-12-31 23:59:59


In [17]:
# Preprocess crime data from BDP Dataset
crime_datetime = []

crime_datetimeofweek  = []
crime_datetimeofyear = []

crime_descript = []
crime_district = []
crime_neighbor = []


# Iterate through all rows, attempt to get datetime columns parsed and working
for index, row in crime_df.iterrows():
    datetime_str = "{} {}".format(row["CrimeDate"], row["CrimeTime"])
    worked_first_parse = False

    try:
        datetime_processed = datetime.strptime(datetime_str, '%m/%d/%Y %H:%M:%S')
        worked_first_parse = True        
    except ValueError as e:
        pass

    if not worked_first_parse:
        try:
            datetime_processed = datetime.strptime(datetime_str, '%m/%d/%Y %H%M')
        except ValueError as e:
            continue

    if start <= datetime_processed <= end:
        n = row["Neighborhood"]
        if isinstance(n, str):
            pd_dt = pd.to_datetime(datetime_processed)
            crime_datetime.append(pd_dt)
            
            crime_datetimeofyear.append(pd_dt.dayofyear)
            crime_datetimeofweek.append(pd_dt.week)
            
            crime_descript.append(row["Description"].lower())
            crime_district.append(row["District"].lower())
            crime_neighbor.append(row["Neighborhood"].lower())
        
# Append to a new dataframe
processed_crime = {
    'Datetime': crime_datetime,
    'Week of the Year': crime_datetimeofweek,
    'Day of the Year': crime_datetimeofyear,
    'Description': crime_descript,
    'District': crime_district,
    'Neighborhood': crime_neighbor
}

processed_crime_df = pd.DataFrame(data=processed_crime)


In [18]:
processed_crime_df.head(10)

,Datetime,Day of the Year,Description,District,Neighborhood,Week of the Year
0,2016-12-31 23:51:00,366,agg. assault,eastern,darley park,52
1,2016-12-31 23:30:00,366,common assault,northwestern,central park heights,52
2,2016-12-31 23:30:00,366,larceny from auto,southeastern,canton,52
3,2016-12-31 23:30:00,366,larceny,eastern,care,52
4,2016-12-31 23:28:00,366,agg. assault,northern,kenilworth park,52
5,2016-12-31 23:15:00,366,burglary,southwestern,irvington,52
6,2016-12-31 23:00:00,366,larceny from auto,northern,charles village,52
7,2016-12-31 23:00:00,366,larceny,northeastern,woodbourne heights,52
8,2016-12-31 22:30:00,366,robbery - street,northeastern,belair-edison,52
9,2016-12-31 22:05:00,366,burglary,southeastern,patterson park neighborho,52


In [19]:
# Generate unique neighborhoods within crime dataset
unique_crime_neighborhoods = np.unique(processed_crime_df["Neighborhood"])

# Begin processing service Dataframe

In [20]:
# sample = 100000
# service_df = pd.read_csv("../datasets/311_Customer_Service_Requests.csv", nrows=sample)
service_df = pd.read_csv("../datasets/311_Customer_Service_Requests.csv")

/Users/jaime/Downloads/CMSC-491-DS-Group-Project/python_processing/.env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
service_df.head(2)

,SRRecordID,ServiceRequesNum,SRType,Agency,Neighborhood,StreetAddress,ZipCode,MethodReceived,SRStatus,CreatedDate,StatusDate,DueDate,LastActivity,Outcome,LastActivityDate,GeoLocation
0,1094344940,15-00072082,WW Water Leak (Exterior),Bureau of Water and Waste Water,CANTON,1013 S CLINTON ST,21224,Interface,CLOSED,02/01/2015 08:12:00 AM +0000,02/01/2015 06:23:00 PM +0000,02/03/2015 08:12:00 AM +0000,Cityworks SR Comments,UPDATED,NaN,NaN
1,1094344982,15-00072083,BCLB-Liquor License Complaint,Liquor License Board,GREEKTOWN,500 S LEHIGH ST,21224,Interface,CLOSED,02/01/2015 08:48:00 AM +0000,02/18/2015 09:12:00 PM +0000,03/21/2015 07:48:00 AM +0000,Dispatch Investigator,Closed,NaN,NaN


In [22]:
start = datetime(2016, 1, 1, 0, 0, 0)
end = datetime(2016, 12, 31, 23, 59, 59)
print(start)
print(end)

2016-01-01 00:00:00
2016-12-31 23:59:59


In [23]:
service_df["CreatedDate"] = pd.to_datetime(service_df["CreatedDate"])

In [24]:
service_df["StatusDate"] = pd.to_datetime(service_df["StatusDate"])

In [25]:
service_type = []
service_agency = []
service_neighborhood = []
service_method_received = []
service_creation_date = []
service_completion_date = []
service_diff_time = []

for index, row in service_df.iterrows():
    sr_status = row["SRStatus"].lower()

    if sr_status == "closed":
        created_date = row["CreatedDate"]
        completion_date = row["StatusDate"]
        diff_in_seconds = abs((completion_date - created_date).total_seconds())
        
        if diff_in_seconds == 0:
            continue
        
        service_type.append(row["SRType"])
        service_agency.append(row["Agency"])
        service_neighborhood.append(row["Neighborhood"].lower())
        service_method_received.append(row["MethodReceived"].lower())
        service_creation_date.append(created_date)
        service_completion_date.append(completion_date)
        service_diff_time.append(diff_in_seconds)
        
processed_service = {
    'Service Requested Type': service_type,
    'Agency': service_agency,
    'Neighborhood': service_neighborhood,
    'Method Received': service_method_received,
    'Creation Date': service_creation_date,
    'Closed Date': service_completion_date,
    'Time Delta in secs': service_diff_time
}

processed_service_df = pd.DataFrame(data=processed_service)

In [26]:
print(len(processed_service_df))
processed_service_df.head(2)

1492553


,Agency,Closed Date,Creation Date,Method Received,Neighborhood,Service Requested Type,Time Delta in secs
0,Bureau of Water and Waste Water,2015-02-01 18:23:00,2015-02-01 08:12:00,interface,canton,WW Water Leak (Exterior),36660.0
1,Liquor License Board,2015-02-18 21:12:00,2015-02-01 08:48:00,interface,greektown,BCLB-Liquor License Complaint,1513440.0


In [27]:
unique_service_neighborhoods = np.unique(processed_service_df["Neighborhood"])


# Begin intersecting data and only using districts that have been defined uniformly

In [28]:
unique_neighborhoods_inter = reduce(np.intersect1d, (unique_income_neighborhoods,
                                    unique_service_neighborhoods, unique_crime_neighborhoods))

print(unique_neighborhoods_inter)
print(len(unique_neighborhoods_inter))


['allendale' 'arlington' 'ashburton' 'barclay' 'beechfield'
 'belair-edison' 'brooklyn' 'canton' 'cedonia' 'cherry hill' 'cheswolde'
 'coldspring' 'curtis bay' 'dickeyville' 'dorchester' 'downtown'
 'druid heights' 'edmondson village' 'federal hill' 'fells point'
 'forest park' 'frankford' 'franklintown' 'guilford' 'hampden'
 'harlem park' 'hawkins point' 'highlandtown' 'hollins market' 'homeland'
 'howard park' 'inner harbor' 'irvington' 'lakeland' 'lauraville'
 'little italy' 'loch raven' 'medfield' 'middle east' 'morrell park'
 'mount washington' 'mount winans' 'oldtown' 'orangeville' 'penn north'
 'poppleton' 'remington' 'reservoir hill' 'sandtown-winchester'
 'seton hill' 'ten hills' 'upton' 'violetville' 'walbrook'
 'west arlington' 'west hills' 'westport' 'woodberry']
58


In [29]:
neighbor_filter_income_df = processed_income_df[processed_income_df["Neighborhood"].isin(unique_neighborhoods_inter)]
neighbor_filter_income_df.head(5)


,Median Household Income,Neighborhood,Total Population
0,37302.17105,allendale,16217
1,37302.17105,irvington,16217
3,53565.07970,beechfield,12264
4,53565.07970,ten hills,12264
5,53565.07970,west hills,12264


In [30]:
neighbor_filter_crime_df = processed_crime_df[processed_crime_df["Neighborhood"].isin(unique_neighborhoods_inter)]
neighbor_filter_crime_df.head(5)



,Datetime,Day of the Year,Description,District,Neighborhood,Week of the Year
2,2016-12-31 23:30:00,366,larceny from auto,southeastern,canton,52
5,2016-12-31 23:15:00,366,burglary,southwestern,irvington,52
8,2016-12-31 22:30:00,366,robbery - street,northeastern,belair-edison,52
11,2016-12-31 21:30:00,366,larceny,western,penn north,52
12,2016-12-31 21:27:00,366,common assault,southern,cherry hill,52


In [31]:
neighbor_filter_service_df = processed_service_df[processed_service_df["Neighborhood"].isin(unique_neighborhoods_inter)]
neighbor_filter_service_df.head(5)

,Agency,Closed Date,Creation Date,Method Received,Neighborhood,Service Requested Type,Time Delta in secs
0,Bureau of Water and Waste Water,2015-02-01 18:23:00,2015-02-01 08:12:00,interface,canton,WW Water Leak (Exterior),36660.0
2,Department of Transportation,2015-02-04 22:41:00,2015-02-01 09:24:00,phone,medfield,TRM-Snow/Icy Conditions,307020.0
6,Mayors Office of Information Technology,2015-02-01 17:31:00,2015-02-01 11:50:00,interface,hollins market,ECC-Miscellaneous Request,20460.0
7,Department of Transportation,2015-02-02 07:39:00,2015-02-01 12:07:00,interface,belair-edison,BGE-StLight(s) Out Rear,70320.0
10,Department of Transportation,2016-10-21 16:05:52,2016-10-21 09:31:09,phone,downtown,TRM-Illegal Sign Removal,23683.0


In [32]:
PROCESSED_DATA_DIR = "../datasets/processed_data"

In [33]:
filename = "income.csv"
file_path = os.path.join(PROCESSED_DATA_DIR, filename)

neighbor_filter_income_df.to_csv(file_path,encoding="utf-8")

In [34]:
filename = "crime.csv"
file_path = os.path.join(PROCESSED_DATA_DIR, filename)

neighbor_filter_crime_df.to_csv(file_path,encoding="utf-8")

In [35]:
filename = "service.csv"
file_path = os.path.join(PROCESSED_DATA_DIR, filename)

neighbor_filter_service_df.to_csv(file_path,encoding="utf-8")